In [222]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from numpy import set_printoptions
np.set_printoptions(suppress=True)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import (confusion_matrix, log_loss, precision_score, recall_score,f1_score, precision_recall_curve,
classification_report, roc_curve)
from sklearn.preprocessing import StandardScaler


Attribute information:
For more information, read [Moro et al., 2011].

   Input variables:
   #### bank client data:
   1 - age (numeric)
   
   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services") 
                                       
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
   
   4 - education (categorical: "unknown","secondary","primary","tertiary")
   
   5 - default: has credit in default? (binary: "yes","no")
   
   6 - balance: average yearly balance, in euros (numeric) 
   
   7 - housing: has housing loan? (binary: "yes","no")
   
   8 - loan: has personal loan? (binary: "yes","no")
   
   #### related with the last contact of the current campaign:
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
   
  10 - day: last contact day of the month (numeric)
  
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  
  12 - duration: last contact duration, in seconds (numeric)
  
   #### other attributes:
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  
  15 - previous: number of contacts performed before this campaign and for this client (numeric)
  
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

  Output variable (desired target):
  
  17 - y - has the client subscribed a term deposit? (binary: "yes","no")


In [223]:
df=pd.read_csv('bank_cleaned.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,1,999,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,1,999,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,1,999,0,unknown,no
3,47,blue_collar,married,unknown,no,1506,yes,no,unknown,5,may,1,999,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,1,999,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,3,999,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,2,999,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,5,184,3,success,yes
45209,57,blue_collar,married,secondary,no,668,no,no,telephone,17,nov,4,999,0,unknown,no


In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  campaign   45211 non-null  int64 
 12  pdays      45211 non-null  int64 
 13  previous   45211 non-null  int64 
 14  poutcome   45211 non-null  object
 15  y          45211 non-null  object
dtypes: int64(6), object(10)
memory usage: 5.5+ MB


In [225]:
df['age'].unique()

array([58, 44, 33, 47, 35, 28, 42, 43, 41, 29, 53, 57, 51, 45, 60, 56, 32,
       25, 40, 39, 52, 46, 36, 49, 59, 37, 50, 54, 55, 48, 24, 38, 31, 30,
       27, 34, 23, 26, 61, 22, 21, 20, 66, 62, 83, 75, 67, 70, 65, 68, 64,
       69, 72, 71, 19, 76, 85, 63, 90, 82, 73, 74, 78, 80, 94, 79, 77, 86,
       95, 81, 18, 89, 84, 87, 92, 93, 88])

In [226]:
# def encodeAge(age):
#     conditions = [age < 40,
#                   age < 50,
#                   age < 60,
#                   age < 80,
#                   age < 90,
#                   age < 100]

#     values = ['30s','40s','50s','60s','70s','80s','90s']

# df['age'] = df['age'].apply(encodeAge)

In [227]:
df['month'].values

array(['may', 'may', 'may', ..., 'nov', 'nov', 'nov'], dtype=object)

In [228]:
df['month']=df['month'].str.replace('jan','0')
df['month']=df['month'].str.replace('feb','1')
df['month']=df['month'].str.replace('mar','2')
df['month']=df['month'].str.replace('apr','3')
df['month']=df['month'].str.replace('may','4')
df['month']=df['month'].str.replace('jun','5')
df['month']=df['month'].str.replace('jul','6')
df['month']=df['month'].str.replace('aug','7')
df['month']=df['month'].str.replace('sep','8')
df['month']=df['month'].str.replace('oct','9')
df['month']=df['month'].str.replace('nov','10')
df['month']=df['month'].str.replace('dec','11')
df['month']=df['month'].astype(int)

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  int64 
 11  campaign   45211 non-null  int64 
 12  pdays      45211 non-null  int64 
 13  previous   45211 non-null  int64 
 14  poutcome   45211 non-null  object
 15  y          45211 non-null  object
dtypes: int64(7), object(9)
memory usage: 5.5+ MB


In [230]:
categories=df.select_dtypes(include=[object])
cate_value={}
for column in categories.columns[0:]:
    cate_value.update({column: df[column].unique() })
#     print(df[column].value_counts())
print(cate_value)

{'job': array(['management', 'technician', 'entrepreneur', 'blue_collar',
       'unknown', 'retired', 'admin', 'services', 'self_employed',
       'unemployed', 'housemaid', 'student'], dtype=object), 'marital': array(['married', 'single', 'divorced'], dtype=object), 'education': array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object), 'default': array(['no', 'yes'], dtype=object), 'housing': array(['yes', 'no'], dtype=object), 'loan': array(['no', 'yes'], dtype=object), 'contact': array(['unknown', 'cellular', 'telephone'], dtype=object), 'poutcome': array(['unknown', 'failure', 'other', 'success'], dtype=object), 'y': array(['no', 'yes'], dtype=object)}


In [231]:
for key, value in cate_value.items():
   print(key, sorted(value))

job ['admin', 'blue_collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self_employed', 'services', 'student', 'technician', 'unemployed', 'unknown']
marital ['divorced', 'married', 'single']
education ['primary', 'secondary', 'tertiary', 'unknown']
default ['no', 'yes']
housing ['no', 'yes']
loan ['no', 'yes']
contact ['cellular', 'telephone', 'unknown']
poutcome ['failure', 'other', 'success', 'unknown']
y ['no', 'yes']


In [232]:
data = {'job': ['admin', 'blue_collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self_employed', 'services', 'student', 'technician', 'unemployed', 'unknown'],
    'marital': ['divorced', 'married', 'single','','','','','','','','',''],
        'education': ['primary', 'secondary', 'tertiary', 'unknown','','','','','','','',''],
       'default': ['no', 'yes','','','','','','','','','',''],
       'housing': ['no', 'yes','','','','','','','','','',''],
        'loan': ['no', 'yes','','','','','','','','','',''],
        'contact': ['cellular', 'telephone', 'unknown','','','','','','','','',''],
        'month': ['jan', 'feb','mar', 'apr','may', 'jun', 'jul', 'aug','sep', 'oct', 'nov', 'dec'],
        'poutcome': ['failure', 'other', 'success', 'unknown','','','','','','','',''],
        'y': ['no', 'yes','','','','','','','','','','']}
  
# Create DataFrame
cate = pd.DataFrame(data)
cate               # this is for knowing which number is for which value after encoding

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,admin,divorced,primary,no,no,no,cellular,jan,failure,no
1,blue_collar,married,secondary,yes,yes,yes,telephone,feb,other,yes
2,entrepreneur,single,tertiary,,,,unknown,mar,success,
3,housemaid,,unknown,,,,,apr,unknown,
4,management,,,,,,,may,,
5,retired,,,,,,,jun,,
6,self_employed,,,,,,,jul,,
7,services,,,,,,,aug,,
8,student,,,,,,,sep,,
9,technician,,,,,,,oct,,


In [257]:
cate.to_csv('cate.csv',index=False)

In [233]:
from sklearn.preprocessing import LabelEncoder
lacoder = LabelEncoder()
cols = ['job','marital','education','default','housing','loan','contact','poutcome','y']

for col in df[cols]:

    df[col]= lacoder.fit_transform(df[col])

In [234]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,4,1,999,0,3,0
1,44,9,2,1,0,29,1,0,2,5,4,1,999,0,3,0
2,33,2,1,1,0,2,1,1,2,5,4,1,999,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,4,1,999,0,3,0
4,33,11,2,3,0,1,0,0,2,5,4,1,999,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,10,3,999,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,10,2,999,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,10,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,10,4,999,0,3,0


In [235]:
df.to_csv('bank_labeled.csv',index=False)